In [1]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install tqdm


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoProcessor, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig  # Import the QLoRA configuration
from torchvision import transforms
from io import BytesIO
from datasets import Dataset,load_dataset,DatasetDict
import zipfile
import os
from tqdm import tqdm
import torchvision.transforms as transforms
from google.colab import files
import shutil
import re

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = "HuggingFaceM4/idefics-9b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"]
)

In [6]:
processor = AutoProcessor.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [7]:
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, quantization_config=bnb_config, device_map="auto")

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/99.2k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00018-of-00019.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

IdeficsForVisionText2Text has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Instantiating IdeficsAttention without passing a `layer_idx` is not recommended and will lead to errors during the forward call if caching is used. Please

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [9]:
def convert_to_rgb(image):
    if image.mode == "RGB":
        return image

    image_rgba = image.convert("RGBA")
    background = Image.new("RGBA", image_rgba.size, (255, 255, 255))
    alpha_composite = Image.alpha_composite(background, image_rgba)
    alpha_composite = alpha_composite.convert("RGB")
    return alpha_composite

def ds_transforms(example_batch):
    image_size = processor.image_processor.image_size
    image_mean = processor.image_processor.image_mean
    image_std = processor.image_processor.image_std

    image_transform = transforms.Compose([
        convert_to_rgb,
        transforms.RandomResizedCrop((image_size, image_size), scale=(0.9, 1.0), interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=image_mean, std=image_std)
    ])

    prompts = []
    for i in range(len(example_batch['entity_name'])):
        entity_name = example_batch['entity_name'][i]
        entity_value = example_batch['entity_value'][i]
        prompts.append(
            [
                example_batch['image_link'][i],
                f"Question: What is the {entity_name}? Answer: The {entity_name} is {entity_value}.</s>",
            ],
        )

    inputs = processor(prompts, transform=image_transform, return_tensors="pt").to(device)
    inputs["labels"] = inputs["input_ids"]
    return inputs


In [64]:
dataset_src = "train.csv"
dataset_origin = load_dataset('csv',
                       data_files=dataset_src
                       )


dset_train_test = (load_dataset('csv',
                        data_files=dataset_src,
                        split='train'
                        )
          .train_test_split(train_size=0.7)
          )




dset_test_val = dset_train_test["test"].train_test_split(train_size=0.75)



dataset = DatasetDict({
                      'train': dset_train_test['train'],
                      'validation': dset_test_val['train'],
                      'test': dset_test_val['test']}
                      )


dataset['train'] = dataset['train'].shuffle().select([i for i in range(2000)])
dataset['validation'] = dataset['validation'].shuffle().select([i for i in range(600)])
dataset['test'] = dataset['test'].shuffle().select([i for i in range(1000)])
dataset

DatasetDict({
    train: Dataset({
        features: ['image_link', 'group_id', 'entity_name', 'entity_value'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['image_link', 'group_id', 'entity_name', 'entity_value'],
        num_rows: 600
    })
    test: Dataset({
        features: ['image_link', 'group_id', 'entity_name', 'entity_value'],
        num_rows: 1000
    })
})

In [60]:
#Load and prepare the data
ds = dataset
train_ds = ds["train"]
eval_ds = ds["validation"]
train_ds.set_transform(ds_transforms)
eval_ds.set_transform(ds_transforms)

In [63]:
model_name = checkpoint.split("/")[1]
config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    target_modules = ["q_proj", "k_proj", "v_proj"],
    lora_dropout = 0.05,
    bias="none"
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 19,750,912 || all params: 8,949,430,544 || trainable%: 0.2207


In [16]:
training_args = TrainingArguments(
    output_dir = f"AmazonVision",
    learning_rate = 2e-4,
    fp16 = True,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 1,
    dataloader_pin_memory = False,
    save_total_limit = 3,
    num_train_epochs = 2,
    evaluation_strategy ="epoch",
    save_strategy = "epoch",
    logging_steps = 5,
    remove_unused_columns = False,
    label_names = ["labels"],
    load_best_model_at_end = True,
    report_to = "none",
    optim = "paged_adamw_8bit",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_ds,
    eval_dataset = eval_ds
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.263400,0.236137
2,0.182400,0.225698


TrainOutput(global_step=1000, training_loss=0.24546025335788726, metrics={'train_runtime': 2295.8755, 'train_samples_per_second': 1.742, 'train_steps_per_second': 0.436, 'total_flos': 6299241761589120.0, 'train_loss': 0.24546025335788726, 'epoch': 2.0})

In [19]:
trainer.save_model()  
shutil.make_archive('AmazonVision', 'zip', training_args.output_dir)
files.download('AmazonVision.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [56]:

# Define the entity unit mapping as per your problem statement
entity_unit_map = {
    "width": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "depth": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "height": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "item_weight": {"milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"},
    "maximum_weight_recommendation": {"milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"},
    "voltage": {"millivolt", "kilovolt", "volt"},
    "wattage": {"kilowatt", "watt"},
    "item_volume": {"cubic foot", "microlitre", "cup", "fluid ounce", "centilitre", "imperial gallon", "pint", "decilitre", "litre", "millilitre", "quart", "cubic inch", "gallon"}
}

# Define a function to format predictions according to the problem statement
def format_prediction(prediction, entity_name):
    # Regex pattern to extract valid numeric values and units
    pattern = r'([\d\.]+)\s*(\w+)'
    match = re.search(pattern, prediction)

    if match:
        value, unit = match.groups()

        # Check if the unit is valid based on the entity name
        if unit in entity_unit_map.get(entity_name, {}):
            try:
                # Ensure the value is a float and return in the correct format
                formatted_value = f"{float(value):.1f}"  # Use standard float formatting
                return f"{formatted_value} {unit}"
            except ValueError:
                # If value can't be converted to float, return empty string
                return ""

    # Return an empty string if no valid prediction was found
    return ""


# Inference function to run on the entire test dataset with a progress bar
def do_inference_on_dataset(model, processor, dataset, batch_size=32, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids if bad_words else None
    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    predictions = []
    actuals = []

    # Use tqdm to show a progress bar
    for i in tqdm(range(0, len(dataset), batch_size), desc="Processing batches"):
        batch = dataset.select(range(i, min(i + batch_size, len(dataset))))  # Select batch

        prompts = []
        for sample in batch:
            entity_name = sample['entity_name']
            entity_value = sample['entity_value']
            image_link = sample['image_link']
            actuals.append(entity_value)

            prompt = f"Question: What is the {entity_name}? Answer: The {entity_name} is {entity_value}.</s>"
            prompts.append([image_link, prompt])

        inputs = processor(prompts, return_tensors="pt").to(device)

        generated_ids = model.generate(
            **inputs,
            eos_token_id=eos_token_id,
            bad_words_ids=bad_words_ids,
            max_new_tokens=max_new_tokens,
            early_stopping=False,  # Set to False for greedy decoding
            num_beams=1  # Change to a higher number if you want beam search
        )

        generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)

        for text in generated_texts:
            answer_part = text.split("Answer:")[-1].strip()
            predictions.append(answer_part)

    return predictions, actuals

test_dataset=dataset["test"]
# Run inference on the test dataset with a progress bar
predictions, actuals = do_inference_on_dataset(model, processor, test_dataset, batch_size=32, max_new_tokens=100)

# Create a DataFrame for comparison
df_results = pd.DataFrame({
    'Predicted': predictions,
    'Actual': actuals
})



Processing batches: 100%|██████████| 32/32 [26:10<00:00, 49.07s/it]


In [57]:
# Prepare the results for submission
submission_results = []

# Create a DataFrame from the test dataset
test_df = pd.DataFrame(test_dataset)

# Add an index column if it doesn't exist
if 'index' not in test_df.columns:
    test_df['index'] = test_df.index

# Iterate over the actual and predicted results
for index, (pred, actual) in tqdm(enumerate(zip(predictions, actuals)), total=len(predictions), desc="Formatting results"):
    entity_name = test_df.iloc[index]['entity_name']
    formatted_prediction = format_prediction(pred, entity_name)  # Format prediction

    # Append index, formatted prediction, and actual value to results
    submission_results.append({
        "index": test_df.iloc[index]['index'],
        "prediction": formatted_prediction,
        "actual": actual  # Add the actual value here
    })

# Create a DataFrame for submission
submission_df = pd.DataFrame(submission_results)

# Save the DataFrame to a CSV file
submission_df.to_csv('test_out.csv', index=False)

# Display the DataFrame to verify
print(submission_df.head())


Formatting results: 100%|██████████| 1000/1000 [00:00<00:00, 5047.69it/s]

   index       prediction           actual
0      0        0.3 metre        0.3 metre
1      1  72.0 centimetre  72.0 centimetre
2      2        15.7 inch        15.7 inch
3      3         1.7 inch         1.7 inch
4      4       110.0 gram       110.0 gram


In [58]:
def calculate_precision_recall(df):
    # Initialize counts
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    # Iterate through the DataFrame
    for index, row in df.iterrows():
        pred = row['prediction'].strip()
        actual = row['actual'].strip()

        if pred == actual and pred != '':
            true_positives += 1
        elif pred != actual and pred != '':
            false_positives += 1
        elif pred == '' and actual != '':
            false_negatives += 1

    # Calculate precision and recall
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    return precision, recall

# Calculate precision and recall
precision, recall = calculate_precision_recall(submission_df)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')


Precision: 0.78
Recall: 0.95
